In [1]:
import pyfixest as pf

import polars as pl
import pandas as pd

import numpy as np
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri

During startup - Warning messages:
1: Setting LC_COLLATE failed, using "C" 
2: Setting LC_TIME failed, using "C" 
3: Setting LC_MESSAGES failed, using "C" 
4: Setting LC_MONETARY failed, using "C" 


In [2]:
utils = importr("utils")

In [3]:
pandas2ri.activate()
did2s = importr("did2s")
stats = importr("stats")
broom = importr("broom")

In [4]:
df_castle = pd.read_stata("https://github.com/scunning1975/mixtape/raw/master/castle.dta")
df_castle = df_castle.astype({"post": bool, "popwt": np.float64})

df_untreated = df_castle.loc[df_castle["post"] == 0, :]

fit_untrt = pf.feols("l_homicide ~ 1 | state + year", data=df_untreated)
fit_untrt_wgt = pf.feols("l_homicide ~ 1 | state + year", data=df_untreated, weights="popwt")

yhat = fit_untrt.predict(newdata=df_castle)
yhat_wgt = fit_untrt_wgt.predict(newdata=df_castle)

# Getting first stage predictions

yhat_nn = fit_untrt.predict()
yhat_wgt_nn = fit_untrt_wgt.predict()


In [5]:
fit_did2s_unwgt = pf.did2s(
    data=df_castle,
    yname="l_homicide",
    first_stage="~ 0 | sid + year",
    second_stage="~ i(post, ref=0)",
    treatment="post",
    cluster="state",
)

fit_did2s_unwgt.summary()

###

Estimation:  DID2S
Dep. var.: l_homicide_hat, Fixed effects: 0
Inference:  CRV1 (GMM)
Observations:  550

| Coefficient                              |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:-----------------------------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| C(post, contr.treatment(base=0))[T.True] |      0.067 |        0.057 |     1.173 |      0.246 | -0.048 |   0.181 |
---
RMSE: 0.183 R2: 0.015 


In [6]:
fit_did2s_wgt = pf.did2s(
    data=df_castle,
    yname="l_homicide",
    first_stage="~ 0 | sid + year",
    second_stage="~ i(post, ref=0)",
    treatment="post",
    cluster="state",
    weights="popwt"
)

fit_did2s_wgt.summary()

###

Estimation:  DID2S
Dep. var.: l_homicide_hat, Fixed effects: 0
Inference:  CRV1 (GMM)
Observations:  550

| Coefficient                              |   Estimate |   Std. Error |   t value |   Pr(>|t|) |   2.5% |   97.5% |
|:-----------------------------------------|-----------:|-------------:|----------:|-----------:|-------:|--------:|
| C(post, contr.treatment(base=0))[T.True] |      0.075 |        0.035 |     2.124 |      0.039 |  0.004 |   0.146 |
---



In [7]:
fit_did2s_wgt.coef().to_numpy()[0]

0.07514145605366061

In [8]:
df_castle.dtypes

state           object
year             int16
sid               int8
cdl            float32
pre2_cdl       float32
                ...   
_Iyear_2007       int8
_Iyear_2008       int8
_Iyear_2009       int8
_Iyear_2010       int8
popwt          float64
Length: 185, dtype: object

In [9]:
np.sqrt(fit_did2s_wgt._vcov[0][0])

0.03537952705576535

In [15]:
fit_r = did2s.did2s(
    data=df_castle,
    yname="l_homicide",
    first_stage=ro.Formula("~ 0 | sid + year"),
    second_stage=ro.Formula("~ i(post, ref = 0)"),
    treatment="post",
    cluster_var="state",
    weights="popwt"
)

R[write to console]: Running Two-stage Difference-in-Differences
 - first stage formula `~ 0 | sid + year`
 - second stage formula `~ i(post, ref = 0)`
 - The indicator variable that denotes when treatment is on is `post`
 - Standard errors will be clustered by `state`




In [16]:
did2s_df = broom.tidy_fixest(fit_r, conf_int=ro.BoolVector([True]))
did2s_df = pd.DataFrame(did2s_df).T

In [17]:
did2s_df

,0,1,2,3,4,5,6
0,post::TRUE,0.075142,0.03538,2.123875,0.034127,0.005646,0.144637
